In [ ]:
from platform import python_version
print("Versão do Python usada: ", python_version())

In [ ]:
#instalação dos pacotes
#! instala o comando no SO
# sempre olhar no pypi.org se exitesm atualizações

#!pip install imdb-sqlite

#!pip install pycountry  #https://pypi.org/project/pycountry/

In [1]:
#documentação nos comentários
import re # https://docs.python.org/3/library/re.html
import time # https://docs.python.org/3/library/time.html
import sqlite3 # https://docs.python.org/3/library/sqlite3.html
import pycountry # https://github.com/flyingcircusio/pycountry 
import numpy as np # https://numpy.org/
import pandas as pd # https://pandas.pydata.org/docs/
import matplotlib as mlp #https://matplotlib.org/
import matplotlib.pyplot as plt # https://matplotlib.org/
import seaborn as sns # https://seaborn.pydata.org/
from sklearn.feature_extraction.text import CountVectorizer # https://scikit-learn.org/stable/modules/feature_extraction.html
import warnings #https://docs.python.org/3/library/warnings.html
warnings.filterwarnings("ignore")
sns.set_theme(style = "whitegrid")

In [ ]:
# baixando o banco de dados imdb
#pode demorar a depender do desempenho da máquina pode demorar
# tempo total demandado: 47 m 
#%time
#!imdb-sqlite

In [ ]:
#limpar banco de dados anterior (utilizado para teste)
#import os
#os.remove("imdb.db") if os.path.exists("imdb.db") else none

In [ ]:
#criando coneção com o banco de dados SQLite
conect = sqlite3.connect("imdb.db")

In [ ]:
#extraindo a lista de tabelas presente no bando de dados 
tabelas = pd.read_sql_query("SELECT NAME AS 'Table_Name' From sqlite_master Where type = 'table'", conect)

In [ ]:
#Verificar o tipo de tabela criada e visualizar o resultado
type(tabelas)
tabelas.head()

In [ ]:
#convester o dataframe a ser utilizado em uma lista e armazenar em uma tabela
tabelas = tabelas["Table_Name"].values.tolist()

In [ ]:
#Para cada lista é necessário extrair o schema de cada uma
for tabela in tabelas:
    consulta = "PRAGMA TABLE_INFO({})".format(tabela)
    resultado = pd.read_sql_query(consulta, conect)
    print("Esquema da tabela", tabela)
    display(resultado)
    print("-"*100)
    print("\n")

# 1 - Quais são as categorias de filmes mais comuns no IBDM?

In [ ]:
#criação da consulta SQL, contando o número de registros da tabela titles, colocando-os em ordem
consulta1 = '''SELECT type, COUNT(*) AS COUNT FROM tables GROUP BY type'''

In [ ]:
#extração do resultados da query
resultado1 = pd.read_sql_query(consulta1,conect)
display(resultado1)

In [ ]:
#inserir o percentual do resultado da tabela anterior
resultado1 = resultado1['percentual'] = (resultado1['COUNT'] / resultado1['COUNT'].sum()*100)

In [ ]:
display(resultado1)

Irá ser feito a criação de um gráfico com os 3 primeiros títulos e o percentual restante

In [ ]:
#soma dos valores do titles de até 5%
#criando um dicionário para armazenar os valores
other = {}
#filtrar percentual dos titles de até 5% e somar o total
other['COUNT'] = resultado[resultado1['percentual'] < 5]['COUNT'].sum()
#gravando o percentual
other['perccentual'] = resultado1[resultado1['percentual'] < 5]['percentual'].sum()
#ajuste do nome
other['type'] = 'others'

In [ ]:
#visualizar
other

In [ ]:
#filtra o resultado do dataframe restante
resultado1 = resultado1[resultado1['percentual'] > 5]

In [ ]:
#adicionar o resultado com o dataframe, ignorando a indexação criada
resultado1 = resultado1.append(other,ignore_index = True)

In [ ]:
#ordenação dos resultados
resultado1 - resultado1.srot_values(by = 'Count', ascending = False)

In [ ]:
resultado1.head()

In [ ]:
#configuração da leganda do gráfico
labels = [str(resultado1['type'][i])+ ' ' +'{'+str(round(resultado1['percentual'][i],2)) + '%' + ']' for i in resultado1.index]

In [ ]:
#plotando o gráfico
#mapa de cores
#https://matplotlib.org/stable/tutorials/colors/colormaps.html
cs = cm.Set3(np.arange(100))

f = plt.figure()

plt.pie(resultado1['COUNT'], labeldistance = 1, radius = 3, colors = cs, wedgeprops = dict(width = 0.8))
plt.legend(labels = labels, Loc = 'Center', prop = {'size':12})
plt.title("Distribuição de títulos",Loc = 'Center', fontdict = {'fontsize':20, 'fontweigth':20})
plt.show()

# 2 - Qual o número de títulos por genêro?

In [ ]:
#criação da consulta SQL selecionando os generos ods títulos
consulta2 =  '''SELECT genres, COUNT(*) AS COUNT FROM titles WHERE type = 'movie' GROUP BY genres'''

In [ ]:
#gravar o resultado da consulta
resultado2 = pd.read_sql_query(consulta2, conect)

In [ ]:
#converter a str para minusculo
resultado2['genres'] = resultado['genres'].str.lower().values

In [ ]:
#remover (dropar) valores ausentes (NA)
temp = resultado2['genres'].dropna()

Usamos o Countvectorizer para converter a coluna de gêneros em um vetor one-hot encoded para contar o número de filmes em cada gênero.

![title](imagens/one-hot.png)

In [ ]:
# Vamos criar um vetor usando expressão regular para filtrar as strings

# https://docs.python.org/3.8/library/re.html
padrao = '(?u)\\b[\\w-]+\\b'

# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
vetor = CountVectorizer(token_pattern = padrao, analyzer = 'word').fit(temp)

In [ ]:
type(vetor)

In [ ]:
# Aplica a vetorização ao dataset sem valores NA na matriz
bag_generos = vetor.transform(temp)

In [ ]:
type(bag_generos)

In [ ]:
# Retorna do vetor criado, iremos puxar os gêneros únicos
generos_unicos =  vetor.get_feature_names()

In [ ]:
# Cria o dataframe (tabela) de gêneros
generos = pd.DataFrame(bag_generos.todense(), columns = generos_unicos, index = temp.index)

In [ ]:
# Visualiza
generos.info()

In [ ]:
# Drop da coluna n
generos = generos.drop(columns = 'n', axis = 0)

In [ ]:
# Calcula o percentual
generos_percentual = 100 * pd.Series(generos.sum()).sort_values(ascending = False) / generos.shape[0]

In [ ]:
# Visualiza
generos_percentual.head(10)

In [ ]:
# Plot
plt.figure(figsize = (16,8))
sns.barplot(x = generos_percentual.values, y = generos_percentual.index, orient = "h", palette = "terrain")
plt.ylabel('Gênero')             
plt.xlabel("\nPercentual de Filmes (%)")
plt.title('\nNúmero (Percentual) de Títulos Por Gênero\n')
plt.show()

# 3 - Qual a mediana de Avaliação dos Filmes por Gênero?

Mediana de avaliação dos filmes

In [ ]:
#consulta SQL para selecionar os generos dos filmes
consulta3 = '''SELECT rating, genres FROM rating JOIN title ON rating.titles_id = titles.title_id where premiered <= 2022 anda typy = 'movie' '''

In [ ]:
resultado3 = pd.read_sql_query(consulta3,conect)

In [ ]:
display(resultado)

In [ ]:
#criação de uma função para retonrar os gêneros
def retorna_generos(df):
    df['genres'] = df['genres'].str.lower().values
    temp = df['genres'].dropna()
    vetor = CountVectorizer(token_pattern = '(?u)\\b[\\w-]+\\b', analyzer = 'word').fit(temp)
    genero_unicos = vetor.get_features_names()
    genero_unicos = [genre for genre in genero_unicos if len(genre >1)]
    return genero_unicos

In [ ]:
#chamar a função anterior
generis_unicos = retorna_generos(resultado3)

In [ ]:
#criação de lista vazia
genero_count = []
genero_rating = []

In [ ]:
#loop para armazenar os dados de cada tipo na lista
for item in generos_unicos:
    
    # Retorna a contagem de filmes por gênero
    consulta = 'SELECT COUNT(rating) FROM ratings JOIN titles ON ratings.title_id=titles.title_id WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\''
    resultado = pd.read_sql_query(consulta, conect)
    genero_count.append(resultado.values[0][0])
  
     # Retorna a avaliação de filmes por gênero
    consulta = 'SELECT rating FROM ratings JOIN titles ON ratings.title_id=titles.title_id WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\''
    resultado = pd.read_sql_query(consulta, conect)
    genero_rating.append(np.median(resultado['rating']))

In [ ]:
# Prepara o dataframe final
df_genero_ratings = pd.DataFrame()
df_genero_ratings['genres'] = generos_unicos
df_genero_ratings['count'] = genero_count
df_genero_ratings['rating'] = genero_rating

In [ ]:
# Visualiza
df_genero_ratings.head(20)

In [ ]:
# Drop do índice 18 (news)
# Não queremos essa informação como gênero
df_genero_ratings = df_genero_ratings.drop(index = 18)

In [ ]:
# Ordena o resultado
df_genero_ratings = df_genero_ratings.sort_values(by = 'rating', ascending = False)

In [ ]:
# Plot

# Figura
plt.figure(figsize = (16,10))

# Barplot
sns.barplot(y = df_genero_ratings.genres, x = df_genero_ratings.rating, orient = "h")

# Textos do gráfico
for i in range(len(df_genero_ratings.index)):
    
    plt.text(4.0, 
             i + 0.25, 
             str(df_genero_ratings['count'][df_genero_ratings.index[i]]) + " filmes")
    
    plt.text(df_genero_ratings.rating[df_genero_ratings.index[i]],
             i + 0.25,
             round(df_genero_ratings["rating"][df_genero_ratings.index[i]],2))

plt.ylabel('Gênero')             
plt.xlabel('Mediana da Avaliação')
plt.title('\nMediana de Avaliação Por Gênero\n')
plt.show()

# 4 - Qual a mediana da Avaliação dos filme em relação ao ano de estreia?


In [ ]:
# Consulta SQL
consulta4 = '''SELECT rating AS Rating, premiered FROM 
            ratings JOIN titles ON ratings.title_id = titles.title_id 
            WHERE premiered <= 2022 AND type = 'movie'
            ORDER BY premiered
            ''' 


In [ ]:
# Resultado
resultado4 = pd.read_sql_query(consulta4, conect)


In [ ]:
display(resultado4)


In [ ]:
# Calculamos a mediana ao longo do tempo (anos) para cada dado da tabela
ratings = []
for year in set(resultado4['premiered']):
    ratings.append(np.median(resultado4[resultado4['premiered'] == year]['Rating']))


In [ ]:
type(ratings)

In [ ]:
ratings[1:10]

In [ ]:
# Lista de anos
anos = list(set(resultado4['premiered']))

In [ ]:
anos[1:10]

In [ ]:
# Plot
plt.figure(figsize = (16,8))
plt.plot(anos, ratings)
plt.xlabel('\nAno')
plt.ylabel('Mediana de Avaliação')
plt.title('\nMediana de Avaliação dos Filmes Em Relação ao Ano de Estréia\n')
plt.show()

## 5- Qual o Número de Filmes Avaliados Por Gênero Em Relação ao Ano de Estréia?

Número de filmes por gênero. Vamos listar os Top 5.


In [ ]:
# Consulta SQL
consulta5 = '''SELECT genres FROM titles ''' 


In [ ]:
# Resultado
resultado5 = pd.read_sql_query(consulta5, conect)

In [ ]:
display(resultado5)

In [ ]:
# Retorna gêneros únicos, modularização
generos_unicos = retorna_generos(resultado5)

In [ ]:
# Visualiza o resultado
generos_unicos

In [ ]:
# Agora fazemos a contagem de cada genero
genero_count = []
for item in generos_unicos:
    consulta = 'SELECT COUNT(*) COUNT FROM  titles  WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\' AND premiered <= 2022'
    resultado = pd.read_sql_query(consulta, conect)
    genero_count.append(resultado['COUNT'].values[0])

In [ ]:
# Prepara o dataframe
df_genero_count = pd.DataFrame()
df_genero_count['genre'] = generos_unicos
df_genero_count['Count'] = genero_count

In [ ]:
# Calcula os top 5, filtrando o que não tiver n
df_genero_count = df_genero_count[df_genero_count['genre'] != 'n']
df_genero_count = df_genero_count.sort_values(by = 'Count', ascending = False)
top_generos = df_genero_count.head()['genre'].values

In [ ]:
# Plot

# Figura
plt.figure(figsize = (16,8))

# Loop e Plot
for item in top_generos:
    consulta = 'SELECT COUNT(*) Number_of_movies, premiered Year FROM  titles  WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\' AND Year <=2022 GROUP BY Year'
    resultado = pd.read_sql_query(consulta, conect)
    plt.plot(resultado['Year'], resultado['Number_of_movies'])

plt.xlabel('\nAno')
plt.ylabel('Número de Filmes Avaliados')
plt.title('\nNúmero de Filmes Avaliados Por Gênero Em Relação ao Ano de Estréia\n')
plt.legend(labels = top_generos)
plt.show()

## 6- Qual o filme com maior tempo de duração?

calculando os percentuals e filme mais longo da história

In [ ]:
# Consulta SQL com o tempo de duração dos títulos que  não te NaN
consulta6 = ''' SELECT runtime_minutes Runtime 
            FROM titles 
            WHERE type = 'movie' AND Runtime != 'NaN'
            ''' 


In [ ]:
# Resultado
resultado6 = pd.read_sql_query(consulta6, conect)

In [ ]:
display(resultado6)

In [ ]:
# Loop para cálculo dos percentis (função range é exclusiva)
for i in range(101): 
    val = i
    perc = round(np.percentile(resultado6['Runtime'].values, val), 2)
    print('{} percentil da duração (runtime) é: {}'.format(val, perc))

In [ ]:
# Refazendo a consulta e retornando o filme com maior duração
consulta6 = ''' SELECT runtime_minutes Runtime, primary_title
            FROM titles 
            WHERE type = 'movie' AND Runtime != 'NaN'
            ORDER BY Runtime DESC
            LIMIT 1
            ''' 


In [ ]:
resultado6 = pd.read_sql_query(consulta6, conect)

In [ ]:
resultado6

'Logistics' é o filme mais longo da história (até este momento), com 857 horas (51420 minutos). Dirigido por Daniel Andersson e Erika Magnusson, o filme acompanha todo o ciclo de vida de um pedômetro (dispositivo de rastreamento de passos) em ordem cronológica reversa.

# 7 - Qual a relação entre duração e gênero?

In [ ]:
# Consulta SQL
consulta7 = '''SELECT AVG(runtime_minutes) Runtime, genres 
            FROM titles 
            WHERE type = 'movie'
            AND runtime_minutes != 'NaN'
            GROUP BY genres
            ''' 


In [ ]:
# Resultado
resultado7 = pd.read_sql_query(consulta7, conect)


In [ ]:
# Retorna gêneros únicos
generos_unicos = retorna_generos(resultado7)

In [ ]:
# Visualiza
generos_unicos

In [ ]:
# Calcula duração por gênero
genero_runtime = []
for item in generos_unicos:
    consulta = 'SELECT runtime_minutes Runtime FROM  titles  WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\' AND Runtime!=\'NaN\''
    resultado = pd.read_sql_query(consulta, conect)
    genero_runtime.append(np.median(resultado['Runtime']))

In [ ]:
# Prepara o dataframe
df_genero_runtime = pd.DataFrame()
df_genero_runtime['genre'] = generos_unicos
df_genero_runtime['runtime'] = genero_runtime


In [ ]:
# Remove índice 18 (news)
df_genero_runtime = df_genero_runtime.drop(index = 18)


In [ ]:
# Ordena os dados
df_genero_runtime = df_genero_runtime.sort_values(by = 'runtime', ascending = False)


In [ ]:
# Plot

# Tamanho da figura
plt.figure(figsize = (16,8))

# Barplot
sns.barplot(y = df_genero_runtime.genre, x = df_genero_runtime.runtime, orient = "h")

# Loop
for i in range(len(df_genero_runtime.index)):
    plt.text(df_genero_runtime.runtime[df_genero_runtime.index[i]],
             i + 0.25,
             round(df_genero_runtime["runtime"][df_genero_runtime.index[i]], 2))

plt.ylabel('Gênero')             
plt.xlabel('\nMediana de Tempo de Duração (Minutos)')
plt.title('\nRelação Entre Duração e Gênero\n')
plt.show()

# 8 - Qual o núero de filmes produzidos por país?

In [ ]:
# Consulta SQL JOIN junta as tabelas atraves de uma id chave
consulta8 = ''' SELECT region, COUNT(*) Number_of_movies FROM 
            akas JOIN titles ON 
            akas.title_id = titles.title_id
            WHERE region != 'None'
            AND type = \'movie\'
            GROUP BY region
            ''' 


In [ ]:
# Resultado
resultado8 = pd.read_sql_query(consulta8, conect)


In [ ]:
display(resultado8)


In [ ]:
# Shape para visualizar o nº de linhas e de colunas
resultado8.shape

In [ ]:
# Número de linhas
resultado8.shape[0]

In [ ]:
# Listas auxiliares
nomes_paises = []
contagem = []

In [ ]:
# Loop para obter o país de acordo com a região
#bloco de tratamento de erros
for i in range(resultado8.shape[0]):
    try:
        coun = resultado8['region'].values[i]
        nomes_paises.append(pycountry.countries.get(alpha_2 = coun).name)
        contagem.append(resultado8['Number_of_movies'].values[i])
    except: 
        continue

In [ ]:
# Prepara o dataframe
df_filmes_paises = pd.DataFrame()
df_filmes_paises['country'] = nomes_paises
df_filmes_paises['Movie_Count'] = contagem


In [ ]:
# Ordena o resultado
df_filmes_paises = df_filmes_paises.sort_values(by = 'Movie_Count', ascending = False)

In [ ]:
# Visualiza
df_filmes_paises.head(10)

In [ ]:
# Plot

# Figura
plt.figure(figsize = (20,8))

# Barplot
sns.barplot(y = df_filmes_paises[:20].country, x = df_filmes_paises[:20].Movie_Count, orient = "h")

# Loop
for i in range(0,20):
    plt.text(df_filmes_paises.Movie_Count[df_filmes_paises.index[i]]-1,
             i + 0.30,
             round(df_filmes_paises["Movie_Count"][df_filmes_paises.index[i]],2))

plt.ylabel('País')             
plt.xlabel('\nNúmero de Filmes')
plt.title('\nNúmero de Filmes Produzidos Por País\n')
plt.show()

# 9 - Quais são os top 10 melhores filmes

In [ ]:
# Consulta SQL
consulta9 = ''' SELECT primary_title AS Movie_Name, genres, rating
            FROM 
            titles JOIN ratings
            ON  titles.title_id = ratings.title_id
            WHERE titles.type = 'movie' AND ratings.votes >= 25000
            ORDER BY rating DESC
            LIMIT 10          
            ''' 


In [ ]:
# Resultado
top10_melhores_filmes = pd.read_sql_query(consulta9, conect)

In [ ]:
display(top10_melhores_filmes)

# 10 - Quais são os top 10 piores filmes?

In [ ]:
# Consulta SQL
consulta10 = '''
            SELECT primary_title AS Movie_Name, genres, rating
            FROM 
            titles JOIN ratings
            ON  titles.title_id = ratings.title_id
            WHERE titles.type = 'movie' AND ratings.votes >= 25000
            ORDER BY rating ASC
            LIMIT 10
            ''' 


In [ ]:
# Resultado
top10_piores_filmes = pd.read_sql_query(consulta10, conect)


In [ ]:
display(top10_piores_filmes)